# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [18]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [19]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'llama3.2'
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [20]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [21]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [22]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [23]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [24]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [25]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [26]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [27]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Wan-AI/Wan2.1-T2V-14B',
 '/microsoft/Phi-4-multimodal-instruct',
 '/deepseek-ai/DeepSeek-R1',
 '/allenai/olmOCR-7B-0225-preview',
 '/perplexity-ai/r1-1776',
 '/models',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/huggingface/ai-deadlines',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces',
 '/datasets/facebook/natural_reasoning',
 '/datasets/SynthLabsAI/Big-Math-RL-Verified',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/allenai/olmOCR-mix-0225',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel'

In [28]:
get_links("https://huggingface.co")

{'links': [{'type': 'About Company', 'url': 'https://huggingface.co'},
  {'type': 'Company Page', 'url': 'https://huggingface.co brand'},
  {'type': 'Careers/Jobs Page',
   'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [31]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [32]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/team'}, {'type': 'Company page (brand)', 'url': 'https://huggingface.co/brand'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Wan-AI/Wan2.1-T2V-14B
Updated
5 days ago
•
151k
•
734
microsoft/Phi-4-multimodal-instruct
Updated
about 19 hours ago
•
23.6k
•
713
deepseek-ai/DeepSeek-R1
Updated
8 days ago
•
4.3M
•
10.7k
allenai/olmOCR-7B-0225-preview
Updated
7 days ago
•
45.9k
•
364
perplexity-ai/r1-1776
Updated
5 days ago
•
34k
•
1.97k
Browse 1M+ models
Spaces
Running
759
759
Wan2.1
💻
Wan: Open and 

In [43]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Also convert in to spanish"

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [44]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [45]:
get_brochure_user_prompt("youtube", "https://youtube.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.youtube.com/about/'}, {'type': 'press page', 'url': 'https://www.youtube.com/about/press/'}, {'type': 'policies page', 'url': 'https://www.youtube.com/about/policies/'}, {'type': 'how it works page', 'url': 'https://www.youtube.com/howyoutubeworks?utm_campaign=ytgen&utm_source=ythp&utm_medium=LeftNav&utm_content=txt&u=https%3A%2F%2Fwww.youtube.com%2Fhowyoutubeworks%3Futm_source%3Dythp%26utm_medium%3DLeftNav%26utm_campaign%3Dytgen'}, {'type': 'creators page', 'url': 'https://www.youtube.com/creators/'}, {'type': 'careers/jobs page', 'url': ''}, {'type': ' careers and jobs were not present hence the empty string has been given.'}]}


MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [46]:
create_brochure("Youtube", "https://youtube.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.youtube.com/about/'}, {'type': 'press release about company', 'url': 'https://www.youtube.com/about/press/'}, {'type': 'copyright information', 'url': 'https://www.youtube.com/about/copyright/'}, {'type': 'contact page for creators', 'url': 'https://www.youtube.com/creators/'}, {'type': 'advertising options', 'url': 'https://www.youtube.com/ads/'}]}


**Brochure de YouTube**

**Sobre nosotros**

[Imágen de la luna]

Welcome to YouTube, la plataforma más grande del mundo para crear, compartir y disfrutar contenido en línea. Nuestra misión es darle a cada persona una voz y mostrarles al mundo.

**Nuestra cultura**

*   Creemos que cada persona tiene un valor único y contribuye a una comunidad más completa.
*   Nuestro equipo es diverso y comprometido con la inclusión y el empoderamiento de las comunidades marginadas y desfavorecidas.
*   Trabajamos juntos para crear un futuro más brillante.

**Nuestras iniciativas**

*   **Crear para cambiar**: Creemos que cada acción cuenta. Que no cambie tu ciudad.  [Tener un propósito].
¡Hazlo por la primera vez!

**¿Quién somos?**

Nacimos en San Bruno, California en septiembre de 2005 y desde entonces hemos cambiado la forma en que nos comunicamos. Estamos aquí para ayudarte a encontrar, descubrir y compartir tus pasiones.

**Puestos disponibles** 

*   [Aplicación]
*   [Consejos]

**Servicios para las empresas**

*   **Anuncios en YouTube**: Conecte con su audiencia objetivo y alcanzará nuevos clientes.
*   **Anuncios de YouTube AdSense**: Monetiza sus videos y atraiga a más visitantes.

**YouTube Culture & Trends**

YouTube da mucho apoyo. El primer canal de YouTuber es My Cat Has More Fans Than I Do, y luego 1st Class Mancunian, The Amazing Phil, y The Scream Queens se convierten en los canales más populares del mundo. En 2006, comienza a ganar popularidad.

*   **Cannibal**: El canal de YouTuber más rentable de 2023 es Cannibal.
*   YouTube crea una plataforma para creadores de podcast llamada **YouTube Podcasts**

**¿Quién es nuestro equipo?** 

*   **Sundar Pichai, CEO**: Ex Director General de la Corporación Alphabet Inc. y actual CEO de Google.
*   **Neelie Kroes**: Vicepresidenta De la Comisión Europea.

**Tendencias actuales**

YouTube sigue creciente en 2023 y la tendencia actual es: *VFX: When Reality Gets Rendered.*

**Conclusión**

YouTube es una plataforma de contenido en línea donde puedes crear, compartir y disfrutar videos. Nuestra misión es darle a cada persona una voz y mostrarles al mundo.

Spanish Version (Versión en Español):

**Brochure de YouTube**

**Sobre nosotros**

[Imágen de la luna]

Bienvenido a YouTube, la plataforma más grande del mundo para crear, compartir y disfrutar contenido en línea. Nuestra misión es darle a cada persona una voz y mostrarles al mundo.

**Nuestra cultura**

*   Creemos que cada persona tiene un valor único y contribuye a una comunidad más completa.
*   Nuestro equipo es diverso y comprometido con la inclusión y el empoderamiento de las comunidades marginadas y desfavorecidas. 
*   Trabajamos juntos para crear un futuro más brillante.



**Nuestras iniciativas**

*   **Crear para cambiar**: Creemos que cada acción cuenta. Que no cambie tu ciudad.

¡Ahora seras parte!

**¿Quiénes somos?**

Nacimos en San Bruno, California en septiembre de 2005 y desde entonces hemos cambiado la forma en que nos comunicamos. Estamos aquí para ayudarte a encontrar, descubrir y compartir tus pasiones.



**Puestos disponibles**

*   [Aplicación]
*   [Consejos]

**Servicios para las empresas**

*   **Anuncios en YouTube**: Conecte con su audiencia objetivo y alcanzará nuevos clientes.
*   **Anuncios de YouTube AdSense**: Monetiza videos.

**YouTube Culture & Trends (Estilo de vida de YouTube)**

YouTube empieza a funcionar como una industria. El primer canal de YouTuber es My Cat Has More Fans Than I Do, y luego 1st Class Mancunian, The Amazing Phil.

*   **Cannibal**: El canal de YouTuber más rentable de 2023 es Cannibal.

**¿Quién es nuestro equipo?**

*   **Sundar Pichai, CEO**: Ex Director General de la Corporación Alphabet Inc. y actual CEO de Google.
*   **Neelie Kroes**: Vicepresidenta De la Comisión Europea.



**Tendencias actuales (Tendencias)**

YouTube sigue creciendo en 2023.

**Conclusión**

YouTube es una plataforma de contenido en línea donde puedes crear, compartir y disfrutar videos.  Nuestra misión es darle a cada persona una voz y mostrarles al mundo.

Estamos aquí para ayudarte a encontrar, descubrir y compartir tus pasiones.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>